In [373]:
import csv
from time import sleep
from selenium.webdriver.common.keys import Keys
import selenium
from selenium.webdriver.common.by import By

def write_password(driver):
    sleep(1.5)
    password = driver.find_element(by=By.XPATH,
                                   value='//input[@autocomplete="current-password"]')
    password.send_keys('Twitter2022$')
    password.send_keys(Keys.RETURN)

def get_tweet_data(post):
    head = post.find_element(By.XPATH, './/div[2]/div[2]/div[1]/div/div/div[1]/div[1]').text.split('\n')

    try:
        link = post.find_element(By.XPATH, './/div[2]/div[2]/div[2]/div[2]//a').get_attribute('href')
    except:
        link = ''

    try:
        time = post.find_element(By.XPATH, './/div[2]/div[2]/div[1]/div/div/div[1]/a/time').get_attribute('datetime')
    except:
        time = ''

    bottom = post.find_element(By.XPATH, './/div[2]/div[2]/div[2]')
    text = post.find_element(By.XPATH,'.//div[2]/div[2]/div[2]/div[1]').text
    nickname = head[1]
    name = head[0]
    reply = bottom.find_element(By.XPATH, './/div[@data-testid="reply"]').get_attribute('aria-label')

    try:
        retweet = bottom.find_element(By.XPATH, './/div[@data-testid="retweet"]').get_attribute('aria-label')
    except:
        retweet = bottom.find_element(By.XPATH, './/div[@data-testid="unretweet"]').get_attribute("aria-label")

    try:
        like = bottom.find_element(By.XPATH, './/div[@data-testid="unlike"]').get_attribute('aria-label')
    except:
        like = bottom.find_element(By.XPATH, './/div[@data-testid="like"]').get_attribute('aria-label')

    return (time, nickname, name, text, link, reply, retweet, like)


options = selenium.webdriver.ChromeOptions()

options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = selenium.webdriver.Chrome('drivers/chromedriver')

driver.get('https://twitter.com/login')
driver.maximize_window()
sleep(2)

/tmp/ipykernel_20181/4287457119.py:51: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = selenium.webdriver.Chrome('drivers/chromedriver')


In [374]:
username = driver.find_element(by=By.XPATH, value='//input[@autocomplete="username"]')
username.send_keys('zat.tyme.su@gmail.com')
username.send_keys(Keys.RETURN)

In [375]:
try:
    write_password(driver)
    sleep(2)
except:
    login = driver.find_element(by=By.XPATH,
                                value='//input[@autocomplete="on"]')
    login.send_keys('ferkin_kz')
    login.send_keys(Keys.RETURN)
    sleep(1)

    write_password(driver)
    sleep(2)

In [376]:
driver.get('https://twitter.com/elonmusk')
sleep(4)

In [489]:
driver.refresh()
sleep(5)
data = []
tweets_ids = set()
last_pos = driver.execute_script('return window.pageYOffset;')
scrolling = True

while scrolling:
    if len(data) > 100:
        scrolling = False

    page_cards = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')
    for card in page_cards[-40:]:
        tweet = get_tweet_data(card)

        if tweet:
            tweet_id = ''
            for i in tweet:
                tweet_id += str(i)

            if tweet_id not in tweets_ids:
                tweets_ids.add(tweet_id)
                data.append(tweet)

    scroll_attempt = 0
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)

        cur_pos = driver.execute_script('return window.pageYOffset;')
        if last_pos == cur_pos:
            scroll_attempt += 1

            if scroll_attempt >= 3:
                driver.execute_script('window.scrollTo(0, -document.body.scrollHeight);')
            else:
                sleep(2)
        else:
            last_pos = cur_pos
            break



In [490]:
from operator import itemgetter

data.sort(key=itemgetter(0), reverse=True)

In [491]:
for i in data:
    print(i[0])

2022-05-01T06:23:15.000Z
2022-04-29T21:37:16.000Z
2022-04-29T21:19:50.000Z
2022-04-29T12:44:05.000Z
2022-04-29T12:28:10.000Z
2022-04-28T20:58:09.000Z
2022-04-28T20:53:49.000Z
2022-04-25T00:30:36.000Z
2022-04-24T14:25:01.000Z
2022-04-24T11:46:48.000Z
2022-04-23T04:07:18.000Z
2022-04-23T03:30:45.000Z
2022-04-22T23:43:11.000Z
2022-04-22T23:39:48.000Z
2022-04-21T20:20:38.000Z
2022-04-21T20:12:46.000Z
2022-04-21T20:09:15.000Z
2022-04-21T18:56:34.000Z
2022-04-21T18:53:55.000Z
2022-04-21T18:50:52.000Z
2022-04-21T18:01:33.000Z
2022-04-21T17:44:09.000Z
2022-04-21T03:09:12.000Z
2022-04-20T03:32:19.000Z
2022-04-19T18:25:05.000Z
2022-04-18T21:32:12.000Z
2022-04-18T14:15:38.000Z
2022-04-08T16:58:08.000Z
2022-04-08T15:35:45.000Z
2022-04-08T15:18:35.000Z
2022-04-08T10:50:44.000Z
2022-04-08T01:43:01.000Z
2022-04-07T13:50:20.000Z
2022-04-07T08:22:22.000Z
2022-04-07T08:20:08.000Z
2022-04-07T06:17:19.000Z
2022-04-07T06:16:04.000Z
2022-04-07T06:14:07.000Z
2022-04-01T14:01:08.000Z
2022-03-28T09:29:09.000Z


In [493]:
with open('musk_twitter.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['date', 'username', 'name', 'text', 'content', 'reply', 'retweet', 'like']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)